In [ ]:
# Performance Analysis of Memory Biased Random Walk (MBRW)
# MBRW is implemented in both Functional, mbrw_functional(), and Object Oriented, mbrw_oo(), paradigm.

# For further information: Yucel et al., 2016, Detection of network communities with memory-biased random walk algorithms,
#                                              Journal of Complex Networks
# M. Yucel, Unpublished PhD Thesis, Memory Biased Random Walk As An Empirical Approach to Indetifying Network Communities

# Author: Dr. Mesut Yucel - 2016

In [ ]:
import pandas as pd
import numpy as np
import itertools
from collections import deque
from operator import itemgetter
from scipy.special import comb
from functools import reduce
import time

In [ ]:
# Simulation Parameters

alpha = 1000
memory_size = 5
sim_step = 100000

In [ ]:
def get_neighbor_data():
    
    
    
    # Defining each node as an object with data members of ID and Neighbors...

    class Node:

        ' The general node class for network nodes. '

        def __init__(self, ID, Neighbors):

            self.ID = ID
            self.Neighbors = Neighbors


        def get_id(self):

            return self.ID


        def get_neighbors(self):

            return self.Neighbors

    
    
    # Read binary interaction data as a Binary Interaction Matrix

    bin_int_mat = pd.read_csv(r'./desktop/bin_int_mat.csv', dtype=np.uint16)
    N = np.max(np.max(bin_int_mat)) + 1

    
    
    # Convert the Binary Interaction Matrix into Node vs. Neighbors dictionary and 
    # a list of node objects with ID and Neighbors..

    neigh_dict = {}
    neighbor_objects = []

    for node_id in range(N):

        neighbor_indices = [bin_int_mat[bin_int_mat[Node] == node_id].index.tolist() for Node in bin_int_mat.columns]
        neighbor_indices = list(itertools.chain.from_iterable(neighbor_indices))

        neighbors = np.append(bin_int_mat.iloc[neighbor_indices]['Node1'], bin_int_mat.iloc[neighbor_indices]['Node2'])
        neighbors = np.setdiff1d(neighbors, node_id)

        neigh_dict[node_id] = np.uint16(neighbors)             # For functional program
        neighbor_objects.append(Node(node_id, neighbors))      # For object oriented program
        
    
    return {'neigh_dict': neigh_dict, 'neighbor_objects': neighbor_objects, 'N': N}

In [ ]:
def mbrw_functional(neigh_dict, N, sim_step):
    
    
    ' Simulation with functional approach. '
    
    
    current_node = np.random.randint(N)
    previous_node = N
    memorized_sequence = N * np.ones(memory_size)
    memorized_sequence[-1] = current_node
    traversed_sequence = deque([current_node])

    
    # Memory Biased Random Walk Simulation

    while len(traversed_sequence) < sim_step:

        potential_next_nodes = neigh_dict[current_node]
        potential_next_nodes = np.setdiff1d(potential_next_nodes, previous_node)     # No reverse back...

        biases = np.ones(len(potential_next_nodes))


        if current_node in memorized_sequence[:-1]:

            biased_indices = np.where(memorized_sequence[:-1] == current_node)
            biased_indices = biased_indices[0] + 1
            biased_nodes = memorized_sequence[biased_indices]

            for node in biased_nodes:
                biases[np.where(potential_next_nodes == node)] = alpha



        # Choosing the next node with Fitness Proportionate Selection

        random_selection = biases.sum() * np.random.rand()
        previous_node = current_node
        current_node = potential_next_nodes[np.argmax(biases.cumsum() > random_selection)]

        traversed_sequence.append(current_node)

        memorized_sequence[:-1] = memorized_sequence[1:]
        memorized_sequence[-1] = current_node
        

In [ ]:
def mbrw_oo(neighbor_objects, N, sim_step):
    
    
    ' Simulation with object oriented approach. '
    
    
    current_node = np.random.randint(N)
    previous_node = N
    memorized_sequence = N * np.ones(memory_size)
    memorized_sequence[-1] = current_node
    traversed_sequence = deque([current_node])

    
    # Memory Biased Random Walk Simulation

    while len(traversed_sequence) < sim_step:
        
        potential_next_nodes = (neighbor_objects[current_node]).get_neighbors()
        potential_next_nodes = np.setdiff1d(potential_next_nodes, previous_node)     # No reverse back...

        biases = np.ones(len(potential_next_nodes))


        if current_node in memorized_sequence[:-1]:

            biased_indices = np.where(memorized_sequence[:-1] == current_node)
            biased_indices = biased_indices[0] + 1
            biased_nodes = memorized_sequence[biased_indices]

            for node in biased_nodes:
                biases[np.where(potential_next_nodes == node)] = alpha



        # Choosing the next node with Fitness Proportionate Selection

        random_selection = biases.sum() * np.random.rand()
        previous_node = current_node
        current_node = potential_next_nodes[np.argmax(biases.cumsum() > random_selection)]

        traversed_sequence.append(current_node)

        memorized_sequence[:-1] = memorized_sequence[1:]
        memorized_sequence[-1] = current_node


In [ ]:
# Performance analysis with %timeit

data = get_neighbor_data()

In [ ]:
%timeit mbrw_functional(data['neigh_dict'], data['N'], sim_step)

In [ ]:
%timeit mbrw_oo(data['neighbor_objects'], data['N'], sim_step)

In [ ]:
# Performance analysis with time.time()

In [ ]:
t1 = time.time()

mbrw_functional(data['neigh_dict'], data['N'], sim_step)

print('Elapsed time: ', time.time() - t1, 'seconds.')


In [ ]:
t1 = time.time()

mbrw_oo(data['neighbor_objects'], data['N'], sim_step)

print('Elapsed time: ', time.time() - t1, 'seconds.')
